In [2]:
print('hello world')

hello world


Importing Libraries

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.functions import when, col

Necessary configuration to connect spark with GCP

In [ ]:
credentials_location = '/home/jom/.keys/creds-dbt.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "/home/jom/lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [ ]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

25/04/08 11:36:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

Reading the csv file 

In [8]:
df = spark.read.csv('gs://dep-project1-bucket/data/unified_cities.csv', header=True)


[Stage 0:>                                                          (0 + 1) / 1]



In [9]:
df.show(5)

25/04/08 11:36:47 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+--------------------+--------------+----+----------+----------+---------------------------------+-----------------------+----------------------+----------------+---------------------+-----------+----------------------------------------------+--------------------+----------------+---------------------------+------------+------------------+------------------+----------------------+------+---------+------------------------+-------------+------------------------+-----------------------------+------------------------+-------------+-------------------------------+--------------------+-------------------------------+------------+---------------+------------------------------+---------------------------------+----------------------------+
|         data_source|    table_name|year|resiliency|innovation|activeestablis

In [ ]:
# Replace '-' with None (null) across all string columns
for column in df.columns:
    df = df.withColumn(
        column,
        when(col(column) == '-', 0).otherwise(col(column))
    )

In [11]:
df.show(5)

+--------------------+--------------+----+----------+----------+---------------------------------+-----------------------+----------------------+----------------+---------------------+-----------+----------------------------------------------+--------------------+----------------+---------------------------+------------+------------------+------------------+----------------------+------+---------+------------------------+-------------+------------------------+-----------------------------+------------------------+-------------+-------------------------------+--------------------+-------------------------------+------------+---------------+------------------------------+---------------------------------+----------------------------+
|         data_source|    table_name|year|resiliency|innovation|activeestablishmentsinthelocality|safetycompliantbusiness|gettingbusinesspermits|socialprotection|accommodationcapacity|roadnetwork|presenceofbusinessandprofessionalorganizations|employmentgener

In [12]:
df.printSchema()

root
 |-- data_source: string (nullable = true)
 |-- table_name: string (nullable = true)
 |-- year: string (nullable = true)
 |-- resiliency: string (nullable = true)
 |-- innovation: string (nullable = true)
 |-- activeestablishmentsinthelocality: string (nullable = true)
 |-- safetycompliantbusiness: string (nullable = true)
 |-- gettingbusinesspermits: string (nullable = true)
 |-- socialprotection: string (nullable = true)
 |-- accommodationcapacity: string (nullable = true)
 |-- roadnetwork: string (nullable = true)
 |-- presenceofbusinessandprofessionalorganizations: string (nullable = true)
 |-- employmentgeneration: string (nullable = true)
 |-- localeconomysize: string (nullable = true)
 |-- financialtechnologycapacity: string (nullable = true)
 |-- productivity: string (nullable = true)
 |-- financialdeepening: string (nullable = true)
 |-- localeconomygrowth: string (nullable = true)
 |-- transportationvehicles: string (nullable = true)
 |-- health: string (nullable = true)

Data type conversion

In [13]:
for column in df.columns[3:]:
    df = df.withColumn(column, col(column).cast("double"))
    print(f'done: {column}')

done: resiliency
done: innovation
done: activeestablishmentsinthelocality
done: safetycompliantbusiness
done: gettingbusinesspermits
done: socialprotection
done: accommodationcapacity
done: roadnetwork
done: presenceofbusinessandprofessionalorganizations
done: employmentgeneration
done: localeconomysize
done: financialtechnologycapacity
done: productivity
done: financialdeepening
done: localeconomygrowth
done: transportationvehicles
done: health
done: education
done: capacityofhealthservices
done: peaceandorder
done: recognitionofperformance
done: informationtechnologycapacity
done: capacityofschoolservices
done: lguinvestment
done: capacitytogeneratelocalresource
done: governmentefficiency
done: compliancetoartacitizenscharter
done: costofliving
done: distancetoports
done: compliancetonationaldirectives
done: presenceofinvestmentpromotionunit
done: availabilityofbasicutilities


In [14]:
trans_df = df.withColumnRenamed('table_name', 'city')\
                    .drop('data_source')

In [15]:
trans_df.columns

['city',
 'year',
 'resiliency',
 'innovation',
 'activeestablishmentsinthelocality',
 'safetycompliantbusiness',
 'gettingbusinesspermits',
 'socialprotection',
 'accommodationcapacity',
 'roadnetwork',
 'presenceofbusinessandprofessionalorganizations',
 'employmentgeneration',
 'localeconomysize',
 'financialtechnologycapacity',
 'productivity',
 'financialdeepening',
 'localeconomygrowth',
 'transportationvehicles',
 'health',
 'education',
 'capacityofhealthservices',
 'peaceandorder',
 'recognitionofperformance',
 'informationtechnologycapacity',
 'capacityofschoolservices',
 'lguinvestment',
 'capacitytogeneratelocalresource',
 'governmentefficiency',
 'compliancetoartacitizenscharter',
 'costofliving',
 'distancetoports',
 'compliancetonationaldirectives',
 'presenceofinvestmentpromotionunit',
 'availabilityofbasicutilities']

In [16]:
mv /home/jom/dep_project1/dez-project/spark_venv/spark-bigquery-with-dependencies_2.12-0.29.0.jar $SPARK_HOME/jars/


mv: cannot stat '/home/jom/dep_project1/dez-project/spark_venv/spark-bigquery-with-dependencies_2.12-0.29.0.jar': No such file or directory


In [ ]:
spark.read.format("bigquery")

Saving to BigQuery

In [19]:
trans_df.write \
    .format("bigquery") \
    .option("table", "dataengineeringproject-454111.production.transformed_dataset") \
    .option("writeMethod", "direct") \
    .mode("overwrite") \
    .save()


25/04/08 11:38:52 WARN DefaultCredentialsProvider: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/.


Saving to the Bucket

In [20]:
trans_df = trans_df.repartition(1)

In [ ]:
trans_df.write \
    .option("header", True) \
    .mode("overwrite") \
    .csv("gs://dep-project1-bucket/data/transformed_data")
